In [12]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import scipy
import numpy as np
import scipy


In [13]:
data = pd.read_csv("./data.csv")
ports = data["Source Port"].unique()
ports = ports[ports !=1883]
ports = ports[ports != 34366]

data


,No.,Time,Source,Destination,Protocol,Length,Source Port,Message Identifier,Destination Port,Info
0,376,43.636973,192.168.15.6,45.179.91.66,MQTT,89,52623,NaN,1883,Connect Command
1,378,43.652859,192.168.15.6,45.179.91.66,MQTT,354,52623,"1,2",1883,"Subscribe Request (id=1) [devices-1], Publish ..."
2,379,43.652835,45.179.91.66,192.168.15.6,MQTT,70,1883,NaN,52623,Connect Ack
3,381,43.668186,45.179.91.66,192.168.15.6,MQTT,71,1883,1,52623,Subscribe Ack (id=1)
4,383,43.684124,45.179.91.66,192.168.15.6,MQTT,340,1883,2,52623,"Publish Message [devices-1], Publish Ack (id=2)"
...,...,...,...,...,...,...,...,...,...,...
3162,41651,832.615490,45.179.91.66,192.168.15.6,MQTT,68,1883,NaN,60325,Ping Response
3163,41653,832.719069,192.168.15.6,45.179.91.66,MQTT,68,39517,NaN,1883,Ping Request
3164,41654,832.736009,45.179.91.66,192.168.15.6,MQTT,68,1883,NaN,39517,Ping Response
3165,41661,833.293009,192.168.15.6,45.179.91.66,MQTT,68,48589,NaN,1883,Ping Request


In [14]:
devices = {}
#  : devices será um dicionario que irá ter uma chave referente ao device e o valor será um dataframe referente aos respectivos devices 
for port in ports: 
  data = data.loc[ ~(data["Message Identifier"] == "1,2")]
  array_publish = data.loc[data["Source Port"] == port].dropna(subset=['Message Identifier'])
  array_publish_ack = data.loc[data["Destination Port"] == port].dropna(subset=['Message Identifier'])
  device = {"publish":array_publish, "publish_ack" : array_publish_ack}
  devices[port] = device

In [15]:
data = {}
df_concatenado = pd.DataFrame()
for port in ports:
    RTT = pd.DataFrame(columns=["Time_publish", "Time_ack"])
    for index, row in devices[port]["publish"].iterrows():
        id = int(row["Message Identifier"])
        time_publish = row["Time"]
        RTT.loc[id, "Size_Payload"] = 246 

        RTT.loc[id, "Time_publish"] = time_publish 

    for index, row in devices[port]["publish_ack"].iterrows():
        id = int(row["Message Identifier"])
        time_publish = row["Time"]
        RTT.loc[id, "Time_ack"] = time_publish
    RTT.loc[id, "Time_ack"] = time_publish
    RTT["RTT"]  =(RTT["Time_ack"] - RTT["Time_publish"] )* 1000
    data[port] = RTT
    df_concatenado = pd.concat([data[port], df_concatenado])
df_concatenado["RTT"].std()
media = df_concatenado["RTT"].mean()
e = 0.05 * media
n = (1.96 * (df_concatenado["RTT"].std()/e))**2
n


511.03394365609876

In [16]:

media = df_concatenado["RTT"].mean()
e = 0.05 * media
n = (1.96 * (df_concatenado["RTT"].std()/e))**2
n

511.03394365609876

# CÁLCULO DO TAMANHO DA AMOSTRA
***
## Variáveis quantitativas e população infinita
***
## Nívéis de confiança e significância 
O **nível de confiança** ($1 - \alpha$) representa a probabilidade de acerto da estimativa. De forma complementar o **nível de significância** ($\alpha$) expressa a probabilidade de erro da estimativa.

O **nível de confiança** representa o grau de confiabilidade do resultado da estimativa estar dentro de determinado intervalo. Quando fixamos em uma pesquisa um **nível de confiança** de 95%, por exemplo, estamos assumindo que existe uma probabilidade de 95% dos resultados da pesquisa representarem bem a realidade, ou seja, estarem corretos.

O **nível de confiança** de uma estimativa pode ser obtido a partir da área sob a curva normal como ilustrado na figura abaixo.


![alt text](https://caelum-online-public.s3.amazonaws.com/1178-estatistica-parte2/01/img007.png)

Em nosso trabalho iremos utilizar um nivel de confiança de 95%, logo abaixo iremos calcular o Z.

Esta variável $Z$ representa o afastamento em desvios padrões de um valor da variável original em relação à média.

In [17]:
#variavel nível de significancia
nivel_de_significancia = 0.05

In [18]:
#Calculando nivel de confiança
nivel_de_confiança = 1 - nivel_de_significancia
nivel_de_confiança

0.95

In [19]:
#Calculando o z com scipy
z = scipy.stats.norm.ppf(0.975)
z

1.959963984540054

Com nosso z calculado podemos seguir adiante

### $$z = 1.959963984540054 $$

Abaixo segue algumas formulas que iremos utilizar para calcular o numero de amostra

## $$n = \left(z\frac{s}{e}\right)^2$$





Onde:

$z$ = variável normal padronizada

$s$ = desvio padrão amostral

$e$ = erro inferencial


### Calculando a desvio padrão $s$

In [20]:
# Calculando o desvio padrão do nosso único dispositivo
desvio_padrao_amostral = df_concatenado["RTT"].std()
desvio_padrao_amostral

19.878979734084925

In [21]:
#Calculando o erro da nossa amostra equivalente a 5%

media = df_concatenado["RTT"].mean()
e = 0.05 * media
e

1.7235555630652752

### Calculando o tamanho da nossa amostra

# Substituindo todos os valores na formula para calcular o numero de amostra.

In [22]:
n = (z * (desvio_padrao_amostral/e))**2
n

511.01516309137475